# Population

In [1]:
import pandas as pd
import numpy as np
import glob

DATA_FOLDER = '../data/population'

In [2]:
populations = pd.read_csv(DATA_FOLDER + "/cities15000.csv", encoding = "ISO-8859-1")
populations['asciiname_lower'] = populations['asciiname'].apply(str.lower)
populations['name_lower'] = populations['name'].apply(str.lower)
# split alternate names to set of names
populations['alternatenames'] = populations['alternatenames'].dropna().apply(lambda x: {y.lower() for y in x.split(",")})
populations.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,...,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,asciiname_lower,name_lower
0,3040051,les Escaldes,les Escaldes,"{escaldes, escaldes-engordany, lai sai si ka e...",42.50729,1.53414,P,PPLA,AD,NaN,...,NaN,NaN,NaN,15853,NaN,1033.0,Europe/Andorra,10/15/08,les escaldes,les escaldes
1,3041563,Andorra la Vella,Andorra la Vella,"{â¢ä_äää©äèä©äèä«â¤ä»ä£, andorra tu...",42.50779,1.52109,P,PPLC,AD,NaN,...,NaN,NaN,NaN,20430,NaN,1037.0,Europe/Andorra,5/30/10,andorra la vella,andorra la vella
2,290594,Umm al Qaywayn,Umm al Qaywayn,"{am alqywyn, umm al qaiwain, oumm al qaìøwaìøn...",25.56473,55.55517,P,PPLA,AE,NaN,...,NaN,NaN,NaN,44411,NaN,2.0,Asia/Dubai,10/7/14,umm al qaywayn,umm al qaywayn
3,291074,Ras al-Khaimah,Ras al-Khaimah,"{ras al khaimah, khaimah, rkt, ras el khaìømah...",25.78953,55.9432,P,PPLA,AE,NaN,...,NaN,NaN,NaN,115949,NaN,2.0,Asia/Dubai,12/5/15,ras al-khaimah,ras al-khaimah
4,291696,Khawr Fakkn,Khawr Fakkan,"{khor fakhan, khor'fakkan, khor fakkan, port k...",25.33132,56.34199,P,PPL,AE,NaN,...,NaN,NaN,NaN,33575,NaN,20.0,Asia/Dubai,10/25/13,khawr fakkan,khawr fakkn


In [3]:
populations.shape

(23469, 21)

### Wrong data

In [4]:
populations[populations['name'] == "52.92277"]['geonameid'].values[0]

'Gorad Dehrbi,Ntermpi,dabi,dar bi,darbi,darby  anglstan,de bi,deobi,drby,dyrby,ë\x9dìãë_ì\x81ë_ìÛë_,_Ò__\x84Û_¡_« _Ó\x84\x8d\x84Û_±\x84Ð,_Ó_¡\x84Û_±_ü,_Ó_µ\x84Û_±_ü,_Ó_µ\x84Û_±\x84Ð,ï«Í´\x85ÛÍ¢ÍÇ,_Ò_\x90_¬_Ô_ª,_Ò_¬_Ô_ª,¯ø¯¤¯±¯¬óÎ¯Î ¯¤ô\xa0òøôã¯_¯»¯¤ô\xa0,¯øô_¯±¯¬ô_,òö¯±¯¬óÎ,\x88_Á\x88_¡\x88´\x8d\x88_Â\x88´Û,\x88_Á\x88__\x88_¡\x88¤\x8d\x88_Â\x88_À,\x88üÓ\x88ü_\x88ü£\x88_Î\x88ü_\x88üµ,\x87Ü¡\x87ö_\x87ä¢,\x8bÄÛ\x8bÄ_\x8bÄÒ\x8bÄ_,\x8c_á¾øÓ,\x91\x8dÓ\x91_ã'

### Load Tomas' data

In [5]:
from gen_regions.regions import Regions

In [6]:
data_tomas = Regions()

In [22]:
regions_filename = DATA_FOLDER + "/../regions.csv"
with open(regions_filename, 'rt') as f:
    data_tomas.read_csv(f)

In [8]:
data_tomas.regions

{'world': <Region 'world' [world]>,
 'africa': <Region 'africa' (world) [continent]>,
 'angola': <Region 'angola' (africa) [country]>,
 'menongue': <Region 'menongue' (angola) [city]>,
 'huambo': <Region 'huambo' (angola) [city]>,
 "m'banza congo": <Region "m'banza congo" (angola) [city]>,
 'soyo': <Region 'soyo' (angola) [city]>,
 'lubango': <Region 'lubango' (angola) [city]>,
 'kuito': <Region 'kuito' (angola) [city]>,
 'cabinda': <Region 'cabinda' (angola) [city]>,
 'luena': <Region 'luena' (angola) [city]>,
 'ondjiva': <Region 'ondjiva' (angola) [city]>,
 'malange': <Region 'malange' (angola) [city]>,
 'saurimo': <Region 'saurimo' (angola) [city]>,
 'luanda': <Region 'luanda' (angola) [city]>,
 'catumbela': <Region 'catumbela' (angola) [city]>,
 'namibe': <Region 'namibe' (angola) [city]>,
 'sao tome and principe': <Region 'sao tome and principe' (africa) [country]>,
 'sao tome': <Region 'sao tome' (sao tome and principe) [city]>,
 'gabon': <Region 'gabon' (africa) [country]>,
 'po

In [9]:
cities_tomas = [x.name for x in data_tomas.regions.values() if x.kind == "city"]

In [10]:
cities_tomas_df = pd.DataFrame(cities_tomas, columns=["name"])
cities_tomas_df.head()

,name
0,menongue
1,huambo
2,m'banza congo
3,soyo
4,lubango


In [11]:
len(cities_tomas)

3225

In [12]:
cities_tomas_df.merge(populations, left_on="name", right_on="asciiname_lower", how="inner")

,name_x,geonameid,name_y,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,...,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,asciiname_lower,name_lower
0,menongue,3347353,Menongue,Menongue,"{äáä_ä_â¡â¬, ___ê_¥_ê_õ_ó, menongñ, vila...",-14.6585,17.69099,P,PPLA,AO,...,NaN,NaN,NaN,32203,NaN,1355.0,Africa/Luanda,6/23/14,menongue,menongue
1,huambo,3348313,Huambo,Huambo,"{wambu, nova lisboa, ôàôö¯¤ôé¯¬ôö¯î ¯¢ô òøôöôã...",-12.77611,15.73917,P,PPLA,AO,...,NaN,NaN,NaN,226145,NaN,1716.0,Africa/Luanda,6/4/13,huambo,huambo
2,lubango,3347762,Lubango,Lubango,"{sdd, sa da bandeira, sìá da bandeira, lwbngw ...",-14.91717,13.4925,P,PPLA,AO,...,NaN,NaN,NaN,102541,NaN,1760.0,Africa/Luanda,6/23/14,lubango,lubango
3,cabinda,2243271,Cabinda,Cabinda,"{cab, cabinda, kabinda, ___¡_±_ü___«_¡}",-5.55,12.2,P,PPLA,AO,...,NaN,NaN,NaN,66020,NaN,1.0,Africa/Luanda,1/17/12,cabinda,cabinda
4,luena,3347719,Luena,Luena,"{£¬ñâ÷, ôãôöô ¯¤¯î ¯¢ô òøôöôã¯¤, ý¤ùä«...",-11.78333,19.91667,P,PPLA,AO,...,NaN,NaN,NaN,21115,NaN,1337.0,Africa/Luanda,1/17/12,luena,luena
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,daru,2098329,Daru,Daru,"{daru government station, darw, äûäç, ¯ø¯¤¯±...",-9.07786,143.20893,P,PPLA,PG,...,NaN,NaN,NaN,15214,NaN,11.0,Pacific/Port_Moresby,4/18/15,daru,daru
2192,marau,3457817,Marau,Marau,NaN,-28.44917,-52.2,P,PPL,BR,...,4311809,NaN,NaN,26860,NaN,553.0,America/Sao_Paulo,8/3/12,marau,marau
2193,honiara,2108502,Honiara,Honiara,"{ôàôöô ô_¯¤¯±¯¤, _âü¨üªüµü_ü_ü£ü_, hon...",-9.43333,159.95,P,PPLC,SB,...,NaN,NaN,NaN,56298,NaN,49.0,Pacific/Guadalcanal,1/18/12,honiara,honiara
2194,arona,2521582,Arona,Arona,"{a luo na, arona, ¯£¯±ôöô ¯¤¯î ¯´¯_¯¬¯¤ô ô_¯¤,...",28.09962,-16.68102,P,PPLA3,ES,...,TF,38006,NaN,78614,NaN,632.0,Atlantic/Canary,9/3/15,arona,arona


In [13]:
cities_tomas_df.merge(populations, left_on="name", right_on="name_lower", how="inner")

,name_x,geonameid,name_y,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,...,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,asciiname_lower,name_lower
0,menongue,3347353,Menongue,Menongue,"{äáä_ä_â¡â¬, ___ê_¥_ê_õ_ó, menongñ, vila...",-14.6585,17.69099,P,PPLA,AO,...,NaN,NaN,NaN,32203,NaN,1355.0,Africa/Luanda,6/23/14,menongue,menongue
1,huambo,3348313,Huambo,Huambo,"{wambu, nova lisboa, ôàôö¯¤ôé¯¬ôö¯î ¯¢ô òøôöôã...",-12.77611,15.73917,P,PPLA,AO,...,NaN,NaN,NaN,226145,NaN,1716.0,Africa/Luanda,6/4/13,huambo,huambo
2,lubango,3347762,Lubango,Lubango,"{sdd, sa da bandeira, sìá da bandeira, lwbngw ...",-14.91717,13.4925,P,PPLA,AO,...,NaN,NaN,NaN,102541,NaN,1760.0,Africa/Luanda,6/23/14,lubango,lubango
3,cabinda,2243271,Cabinda,Cabinda,"{cab, cabinda, kabinda, ___¡_±_ü___«_¡}",-5.55,12.2,P,PPLA,AO,...,NaN,NaN,NaN,66020,NaN,1.0,Africa/Luanda,1/17/12,cabinda,cabinda
4,luena,3347719,Luena,Luena,"{£¬ñâ÷, ôãôöô ¯¤¯î ¯¢ô òøôöôã¯¤, ý¤ùä«...",-11.78333,19.91667,P,PPLA,AO,...,NaN,NaN,NaN,21115,NaN,1337.0,Africa/Luanda,1/17/12,luena,luena
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1965,goroka,2096742,Goroka,Goroka,"{goroka, gka, garoka, new garoka}",-6.08336,145.38735,P,PPLA,PG,...,NaN,NaN,NaN,18503,NaN,1581.0,Pacific/Port_Moresby,1/16/12,goroka,goroka
1966,daru,2098329,Daru,Daru,"{daru government station, darw, äûäç, ¯ø¯¤¯±...",-9.07786,143.20893,P,PPLA,PG,...,NaN,NaN,NaN,15214,NaN,11.0,Pacific/Port_Moresby,4/18/15,daru,daru
1967,marau,3457817,Marau,Marau,NaN,-28.44917,-52.2,P,PPL,BR,...,4311809,NaN,NaN,26860,NaN,553.0,America/Sao_Paulo,8/3/12,marau,marau
1968,honiara,2108502,Honiara,Honiara,"{ôàôöô ô_¯¤¯±¯¤, _âü¨üªüµü_ü_ü£ü_, hon...",-9.43333,159.95,P,PPLC,SB,...,NaN,NaN,NaN,56298,NaN,49.0,Pacific/Guadalcanal,1/18/12,honiara,honiara


# Try locating missing populations

In [14]:
missing_populations = set(cities_tomas) - set(populations["asciiname_lower"])
len(missing_populations)

1330

In [15]:
def get_intersection(alternatenames):
    if isinstance(alternatenames, str):
        intersect = alternatenames.intersection(missing_populations)
        if len(intersect) > 1:
            print(intersect)

        if len(intersect) == 1:
            return intersect.pop()
        else:
            return np.nan
    else:
        return np.nan
    
# populations['alternatenames'] = populations['alternatenames'].dropna().apply(get_intersection).dropna()

In [16]:
intersect = populations['alternatenames'].apply(get_intersection)
populations['asciiname_lower'][~intersect.isna()] = intersect.dropna()

/home/hrosspet/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
cities_tomas_df_1 = cities_tomas_df.merge(populations, left_on="name", right_on="asciiname_lower", how="left")
cities_tomas_df_1.head()

,name_x,geonameid,name_y,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,...,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date,asciiname_lower,name_lower
0,menongue,3347353,Menongue,Menongue,"{äáä_ä_â¡â¬, ___ê_¥_ê_õ_ó, menongñ, vila...",-14.6585,17.69099,P,PPLA,AO,...,NaN,NaN,NaN,32203,NaN,1355.0,Africa/Luanda,6/23/14,menongue,menongue
1,huambo,3348313,Huambo,Huambo,"{wambu, nova lisboa, ôàôö¯¤ôé¯¬ôö¯î ¯¢ô òøôöôã...",-12.77611,15.73917,P,PPLA,AO,...,NaN,NaN,NaN,226145,NaN,1716.0,Africa/Luanda,6/4/13,huambo,huambo
2,m'banza congo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,soyo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,lubango,3347762,Lubango,Lubango,"{sdd, sa da bandeira, sìá da bandeira, lwbngw ...",-14.91717,13.4925,P,PPLA,AO,...,NaN,NaN,NaN,102541,NaN,1760.0,Africa/Luanda,6/23/14,lubango,lubango


### Load Tomas' orig population dataset

In [23]:
world_cities = pd.read_csv(DATA_FOLDER + "/worldcities.csv")
world_cities['city_ascii_lowercase'] = world_cities['city_ascii'].apply(str.lower)
world_cities.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,city_ascii_lowercase
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764,tokyo
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922.0,1840034016,new york
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881,mexico city
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629,mumbai
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519,sao paulo


In [24]:
len(set(world_cities['city_ascii_lowercase']).intersection(missing_populations))

339

In [27]:
cities_tomas_df_1 = cities_tomas_df_1.merge(
    world_cities[world_cities['city_ascii_lowercase'].isin(missing_populations)],
    left_on="name_x",
    right_on="city_ascii_lowercase",
    how="left"
)
cities_tomas_df_1.head()

,name_x,geonameid,name_y,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,...,lat,lng,country,iso2,iso3,admin_name,capital,population_y,id,city_ascii_lowercase
0,menongue,3347353,Menongue,Menongue,"{äáä_ä_â¡â¬, ___ê_¥_ê_õ_ó, menongñ, vila...",-14.6585,17.69099,P,PPLA,AO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,huambo,3348313,Huambo,Huambo,"{wambu, nova lisboa, ôàôö¯¤ôé¯¬ôö¯î ¯¢ô òøôöôã...",-12.77611,15.73917,P,PPLA,AO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,m'banza congo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,soyo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,lubango,3347762,Lubango,Lubango,"{sdd, sa da bandeira, sìá da bandeira, lwbngw ...",-14.91717,13.4925,P,PPLA,AO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cities_tomas_df_1['population'] = cities_tomas_df_1.apply(lambda x: x['population_y'] if np.isnan(x['population_x']) else x['population_x'], axis=1)

In [ ]:
cities_tomas_df_1.apply(lambda x: x['population_y'] if np.isnan(x['population_x']) else x['population_x'], axis=1)

### Find if new cities in new datasets

In [29]:
missing_tomas = set(cities_tomas) - set(world_cities['city_ascii_lowercase'])
len(missing_tomas)

1227

In [30]:
len(missing_tomas - set(populations['asciiname_lower']))

991

### Update values in regions

In [ ]:
city = 'andahuaylas'
pop = populations[populations['asciiname_lower'] == city]
# TODO: parse the whole Series
data_tomas.regions[city].pop = int(pop['population'].values[0])
with open(DATA_FOLDER + "/../regions.csv", w) as f:
    data_tomas.write_csv(f)

## Explore structure of Regions

In [32]:
[x.name for x in data_tomas.regions.values() if x.kind == "country"]

['angola',
 'sao tome and principe',
 'gabon',
 'cameroon',
 'chad',
 'congo, dem. rep.',
 'congo, rep.',
 'equatorial guinea',
 'central african republic',
 'algeria',
 'libya',
 'morocco',
 'tunisia',
 'sudan',
 'niger',
 'ghana',
 'cape verde',
 'liberia',
 'burkina faso',
 'mauritania',
 'sierra leone',
 'guinea',
 'benin',
 'togo',
 'nigeria',
 'senegal',
 'gambia',
 "cote d'ivoire",
 'guinea bissau',
 'mali',
 'namibia',
 'botswana',
 'south africa',
 'lesotho',
 'swaziland',
 'mayotte',
 'tanzania',
 'ethiopia',
 'djibouti',
 'seychelles',
 'burundi',
 'zimbabwe',
 'mozambique',
 'mauritius',
 'reunion',
 'comoros',
 'madagascar',
 'rwanda',
 'zambia',
 'kenya',
 'south sudan',
 'somalia',
 'eritrea',
 'malawi',
 'uganda',
 'bangladesh',
 'pakistan',
 'nepal',
 'sri lanka',
 'afghanistan',
 'india',
 'bhutan',
 'maldives',
 'azerbaijan',
 'iran',
 'qatar',
 'jordan',
 'saudi arabia',
 'lebanon',
 'yemen',
 'syria',
 'israel',
 'turkey',
 'oman',
 'kuwait',
 'cyprus, northern',
 

In [33]:
[x.name for x in data_tomas.regions.values() if x.kind == "region"]

['middle east', 'european union']